# Duck Typing

## Example 1: `UnitRange`

In [1]:
x = 1:30

1:30

In [2]:
typeof(x)

UnitRange{Int64}

In [3]:
typeof(x) <: AbstractArray

true

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [4]:
x[3]

3

In [5]:
size(x)

(30,)

In [6]:
eltype(x)

Int64

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [7]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code

In [8]:
@which UnitRange(1, 30)

UnitRange(start::T, stop::T) where T<:Real in Base at range.jl:396

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [10]:
@time collect(1:10000000);

  0.070209 seconds (2 allocations: 76.294 MiB, 5.72% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [11]:
@time 1:10000000;

  0.000001 seconds


In [12]:
M = rand(10,20)

10×20 Matrix{Float64}:
 0.497411  0.0771932  0.432922   0.0125542  …  0.272194  0.142412    0.850949
 0.742695  0.413155   0.0296417  0.431553      0.976545  0.735004    0.597978
 0.308811  0.284344   0.319041   0.255817      0.757814  0.53766     0.901473
 0.699622  0.926508   0.580285   0.147097      0.546256  0.588554    0.213823
 0.351604  0.52716    0.837359   0.788975      0.857022  0.982607    0.651095
 0.773759  0.0509445  0.226297   0.755522   …  0.924178  0.360392    0.718411
 0.647654  0.986879   0.620618   0.951352      0.730518  0.260186    0.206762
 0.36168   0.102241   0.57148    0.694157      0.769257  0.310982    0.435605
 0.302721  0.594742   0.648845   0.16063       0.916544  0.00354482  0.99503
 0.761253  0.276897   0.426285   0.0796682     0.740971  0.973242    0.336694

In [17]:
Mx,My = size(M)

(10, 20)

Yet, in code they **behave** in the same way.

In [18]:
Mx

10

In [19]:
My

20

### Example 2: Diagonal matrix

Let's create a simple custom `DiagonalMatrix` type that can represent square diagonal matrices, i.e.

$$ D = \left( \begin{matrix} x & 0 & 0 & 0 \\ 0 & y & 0 & 0 \\ 0 & 0 & z & 0 \\ 0 & 0 & 0 & \ddots \end{matrix} \right) $$

In [20]:
struct DiagonalMatrix{T} <: AbstractArray{T,2}
    diag::Vector{T}
end

In the spirit of duck typing, we integrate our `DiagonalMatrix` into Julia's type hierarchy by making it a subtype (`<:`) of `AbstractMatrix` to indicate **array-like behavior**. (Note that this does not indicate inheritence of structure!)

Of course, to actually make it behave like a matrix (a two-dimensional array) we must also implement (parts of) the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1).

In [21]:
# implement AbstractArray interface
function Base.getindex(D::DiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

function Base.setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

In [22]:
D = DiagonalMatrix([1,2,3])

3×3 DiagonalMatrix{Int64}:
 1  0  0
 0  2  0
 0  0  3

Note how it's automagically pretty printed (despite the fact that we never defined any special printing)!

In [23]:
D[2,2]

2

In [24]:
D[1,2]

0

In [25]:
size(D)

(3, 3)

In [26]:
D[3,3] = 5

5

In [27]:
D

3×3 DiagonalMatrix{Int64}:
 1  0  0
 0  2  0
 0  0  5

But that's not it. Because of duck typing, all kinds of different functions now "just work".

In [28]:
eltype(D) # element data type

Int64

In [29]:
D + D # addition

3×3 Matrix{Int64}:
 2  0   0
 0  4   0
 0  0  10

In [30]:
D * D # multiplication

3×3 Matrix{Int64}:
 1  0   0
 0  4   0
 0  0  25

In [31]:
inv(D) # inversion

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.2

In [32]:
sin.(D) # broadcasting

3×3 Matrix{Float64}:
 0.841471  0.0        0.0
 0.0       0.909297   0.0
 0.0       0.0       -0.958924

In [33]:
using LinearAlgebra
eigen(D) # eigensolver

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 1.0
 2.0
 5.0
vectors:
3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Of course, so far, these operations have suboptimal performance because they don't utilize the special structure inherent to our `DiagonalMatrix` but fall back to generic implementations.

In [34]:
@which D + D

+(A::AbstractArray, B::AbstractArray) in Base at arraymath.jl:6

In [35]:
Base.:+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)

In [36]:
@which D + D

+(Da::DiagonalMatrix, Db::DiagonalMatrix) in Main at In[35]:1

Important note: **user defined types are just as good as built-in types!**

There is nothing special about built-in types. In fact, [they are implemented in essentially the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our `DiagonalMatrix` type does not come with any performance overhead by benchmarking it in a simple function.

## Benchmarking with [`BenchmarkTools.jl`](https://github.com/JuliaCI/BenchmarkTools.jl)

Benchmarking is difficult to do right for many reasons
* computers are noisy machines
* global vs local scope
* the first function call is special in Julia (more later)
* ...

In [37]:
g(x) = x + 2*x

g (generic function with 1 method)

In [39]:
x = rand(2,2)
@time g.(x)

  0.000014 seconds (3 allocations: 128 bytes)


2×2 Matrix{Float64}:
 0.316001  1.53853
 0.244651  1.54946

In [40]:
function f()
    x = rand(2,2)
    @time g.(x)
end

f (generic function with 1 method)

In [41]:
f()

  0.000001 seconds (1 allocation: 96 bytes)


2×2 Matrix{Float64}:
 2.48064  0.39683
 1.50389  2.49252

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

General rule: **Don't use `@time` but `@btime`** from [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) and interpolate (`$`) input arguments.

In [42]:
using BenchmarkTools

In [43]:
a = "test"

"test"

In [44]:
"This is a $a !"

"This is a test !"

In [45]:
@btime g.($x)

  61.096 ns (1 allocation: 96 bytes)


2×2 Matrix{Float64}:
 0.316001  1.53853
 0.244651  1.54946

In [46]:
@benchmark g.($x)

BenchmarkTools.Trial: 10000 samples with 979 evaluations.
 Range (min … max):  63.433 ns … 985.362 ns  ┊ GC (min … max): 0.00% … 90.10%
 Time  (median):     66.676 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.134 ns ±  33.352 ns  ┊ GC (mean ± σ):  2.05% ±  3.94%

    ▂▅▇█▇▆▄▂▁     ▁▃▃▂                                         ▂
  ▄▇████████████▇▆█████▆▇▅▅▅▄▆▄▅▅▆▃▆▄▄▅▅▄▄▄▄▃▅▃▁▁▃▃▄▃▄▄▆▄▅▃▄▄▄ █
  63.4 ns       Histogram: log(frequency) by time      97.1 ns <

 Memory estimate: 96 bytes, allocs estimate: 1.

### Custom types are just as good as built-in types

Let's compare our custom `DiagonalMatrix` against the standard `Diagonal` type that ships in the `LinearAlgebra` standard library.

In [47]:
using LinearAlgebra
x = rand(100);
Djl = Diagonal(x)
D = DiagonalMatrix(x)
@btime $Djl + $Djl;
@btime $D + $D;

  94.568 ns (1 allocation: 896 bytes)
  94.858 ns (1 allocation: 896 bytes)


# Core messages of this notebook

* Duck typing is about **shared behavior** instead of shared structure.
* **User defined types are as good as built-in types.**
* We can **extend Base functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* Functions should almost always be benchmarked with **BenchmarkTools.jl's `@btime` and `@benchmark`** instead of `@time`.